In [1]:
from pyinaturalist import *
import pandas as pd
import arcpy

arcpy.env.overwriteOutput = True

#### Import observations from inat API

In [135]:
# Change these every time you run it
# name = 'redmaids'
# code = 'CAME'
# species = 'Calandrinia menziesii'
# taxon = 400267

# name = 'yarrow'
# code = 'ACMI'
# species = 'Achillea millefolium' 
# taxon = 52821

# name = 'blackoak'
# code = 'QUKE'
# species = 'Quercus kelloggii'
# taxon = 49919

# name = 'bluewildrye'
# code = 'ELGL'
# species = 'Elymus glaucus'
# taxon = 52791

# name = 'brome'
# code = 'BRCA'
# species = 'Bromus carinatus'
# taxon = 52700

name ='allvascularplants'
code = 'PLAN'
species = 'All plants'
taxon = 211194

# name = 'yerbabuena'
# code = 'CLDO'
# species = 'Clinopodium douglasii'
# taxon = 53203

# name = 'soaproot'
# code = 'CHPO'
# species = 'Chlorogalum pomeridianum'
# taxon = 47597

# name = 'hazelnut'
# code = 'COCO'
# species = 'Corylus'
# taxon = 53368

# name = 'thimbleberry'
# code = 'RUPA'
# species = 'Rubus parviflorus'
# taxon = 51646

# name = 'blackberry'
# code = 'RUUR'
# species = 'Rubus ursinus'
# taxon = 53445

# name = 'blueelderberry'
# code = 'SACE'
# species = 'Sambucus nigra' # this is what AMLT uses
# taxon = 143799

# name = 'huckleberry'
# code = 'VAOV'
# species = 'Vaccinium ovatum'
# taxon = 48349

# name = 'mugwort'
# code = 'ARDO'
# species = 'Artemisia douglasiana'
# taxon = 52854

# name = 'manzanitas'
# code = 'ARCT'
# species = 'Arctostaphylos'
# taxon = 47179

# Find the taxon id on wikipedia or https://www.inaturalist.org/taxa/47126-Plantae
# taxon = 69823 # tan oak
# taxon = 49011 # valley oak

# Specify criteria for observations
# https://pyinaturalist.readthedocs.io/en/latest/modules/pyinaturalist.v1.observations.html#pyinaturalist.v1.observations.get_observations
    # used to be for a larger/rougher area
    # response = get_observations(taxon_id=taxon, swlat = 35.7880, swlng = -122.5182, nelat = 37.7114, nelng = -120.2098, page='all', per_page=200, quality_grade='research')
# area = SDM.gdb/BoundingBox, roughly the AMLT stewardship area
response = get_observations(taxon_id=taxon, swlat = 36.077540, swlng = -122.605514, nelat = 37.381013, nelng = -120.292403, page='all', per_page=200, quality_grade='research')
# response

This request is larger than recommended for API usage. For bulk requests, consider using the iNat export tool instead: https://www.inaturalist.org/observations/export


MemoryError: 

In [124]:
# Select results from JSON Data, and normalize them
# This flattens the nested iNat data structure and converts to pandas dataframe
data = response['results']
df = pd.json_normalize(data)

# Split the location data into latitude and Longitude columns
# this is the format of location col [38.8418469038, -123.0608224869]
df['lat'] = df['location'].apply(lambda x:x[0])
df['lon'] = df['location'].apply(lambda x:x[1])

# Subset dataframe to include only the fields of interest
df = df[['lat','lon','quality_grade','time_observed_at','scientific_name','common_name','positional_accuracy','public_positional_accuracy']]

# Show the first 8 lines of the dataframe below cell
df.head(8)

,lat,lon,quality_grade,time_observed_at,species_guess,positional_accuracy,public_positional_accuracy
0,37.181496,-121.845535,research,2009-12-31T14:27:00-08:00,Big Berry Manzanita,NaN,NaN
1,37.164092,-122.063336,research,None,Bonny Doon manzanita,NaN,28447.0
2,37.111142,-122.126218,research,2011-05-08T09:45:49-07:00,Santa Cruz Manzanita,NaN,28447.0
3,37.003917,-122.137798,research,2011-05-08T09:52:17-07:00,Bonny Doon manzanita,NaN,28447.0
4,37.055492,-122.143636,research,2011-05-08T10:05:05-07:00,Brittleleaf Manzanita,NaN,NaN
5,37.055561,-122.143207,research,2011-05-08T10:06:39-07:00,glossyleaf manzanita,NaN,NaN
6,37.147297,-121.776260,research,2011-09-03T10:19:18-07:00,Big Berry Manzanita,5.0,5.0
7,36.665211,-121.735924,research,None,toro manzanita,NaN,28505.0


In [125]:
# This is a user-defined function to push dataframe into arcgis feature class
def addData(fc, occurrences):
    arcpy.management.DeleteRows(fc)
    cursor = arcpy.da.InsertCursor(fc, ["SHAPE@XY","lat","lon","quality_grade", "time_observed_at","species_guess","positional_accuracy","public_positional_accuracy"])

    for index, row in occurrences.iterrows():
        xy = (row['lon'],row['lat'])
        cursor.insertRow([xy,row['lat'], row['lon'], row['quality_grade'], row['time_observed_at'], row['species_guess'], row['positional_accuracy'],row['public_positional_accuracy']])
    del cursor

#### Add iNat data to a feature class

In [126]:
# Define your workspace and geodatabase
# workspace = "D:/1_AMLT/1_SDM/SDM_Pro/"
workspace = "D:/1_AMLT/2_Ethnobotanical_Data_Pro/iNaturalist/"

# If file geodatabase already exists, name of file geodatabase
filegdbname = "iNat.gdb"

# # if file geodatabase does not exist, creates file geodatabase
filegdb = "%s%s" % (workspace, filegdbname)
# if (arcpy.Exists(filegdb)) == False:
#     arcpy.management.CreateFileGDB(workspace, filegdbname[:-4], "CURRENT")

# We are creating a feature class within the geodatabase to add the iNat data to
fc = name # "blackoak"
fcPath = "%s/%s" % (filegdb, fc)

# If feature class does not exist, create feature class
if (arcpy.Exists(fcPath)) == False:
    arcpy.management.CreateFeatureclass(filegdb, fc, "POINT", None, "DISABLED", "DISABLED", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision', '', 0, 0, 0, '')

    # Fields to create in feature class
    fields = ("lat DOUBLE # # # #;" +
        "lon DOUBLE # # # #;" +
        "quality_grade TEXT # 255 # #;" +
        "time_observed_at TEXT # 255 # #;" +
        "species_guess TEXT # 255 # #;" +
        "positional_accuracy DOUBLE # # # #;" +
        "public_positional_accuracy DOUBLE # # # #")

    arcpy.management.AddFields(name, fields) 


In [127]:
print(code)
# Add iNat data from the data frame to the newly created feature class
# Note: You will have to remove and then add the point feature class again in the Map to show the data. 
addData(fcPath, df)

ARCT


### Processing iNat data for SAHM

#### Project and clip iNat data

In [128]:
print(code)
# Project to my SDM
output = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/iNat_" + code

area = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/BoundingBox" 
outsr = arcpy.Describe(area).spatialReference

arcpy.management.Project(fcPath, output, outsr)

# # Clip to study area boundary - don't need to do this anymore
# inat = output + "_SDM"
# arcpy.analysis.Clip(output, area, inat)

# print number of records/points
result = arcpy.GetCount_management(output)
count = int(result.getOutput(0))
print('Total features in iNat database = ', count)

ARCT
Total features in iNat database =  3552


#### Get any points from my ethnobot data

In [129]:
print(code)
# ethnobot = "D:/1_AMLT/2_Ethnobotanical_Data_Pro/Ethnobotanical_Data.gdb/All_EthnobotData_Merged_012521"
# this layer has to be in the active map
# select from the layer by species name
ethnobot = "All_EthnobotData_Merged_012521"

search = "Species LIKE '%" + species + "%'"
print(search)

arcpy.management.SelectLayerByAttribute(ethnobot, "NEW_SELECTION", search)

# export data
gdb = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/"
out_name = "My_" + code + "_forSAHM"

print('Creating ' + out_name)
arcpy.conversion.FeatureClassToFeatureClass(ethnobot, gdb, out_name)

# clear selection
arcpy.management.SelectLayerByAttribute(ethnobot, "CLEAR_SELECTION")

# print number of records/points
result = arcpy.GetCount_management(out_name)
count = int(result.getOutput(0))
print('Total features in ethnobot database = ', count)

ARCT
Species LIKE '%Arctostaphylos%'
Creating My_ARCT_forSAHM
Total features in ethnobot database =  7


#### Merge iNat and ethnobot data All_QUKE_forSAHM
and number of total points

In [130]:
print(code)
# merge the two into one in SDM.gdb, saved as 'All_****_forSAHM'
inat = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/iNat_" + code
ethno = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/My_" + code + "_forSAHM"
allpoints = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/All_" + code + "_forSAHM"

print('Merging inat and ethnobot db into one output')
arcpy.management.Merge([inat, ethno], allpoints)

# print number of records/points
result = arcpy.GetCount_management(allpoints)
count = int(result.getOutput(0))
print('Total features in combined layer = ', count)
# this fc is ready to use, already in WGS84

ARCT
Merging inat and ethnobot db into one output
Total features in combined layer =  3559


#### Add field (code) and set to 1

In [131]:
print(code)
# allpoints = "D:/1_AMLT/1_SDM/SDM_Pro/SDM.gdb/All_" + code + "_forSAHM"
# add field named as species code
arcpy.AddField_management(allpoints, code, "LONG")
# set to one
arcpy.management.CalculateField(allpoints, code, 1)
print('Field added')

ARCT
Field added


#### Point to raster (env = template raster for prj, cell size, snapping, extent) All_code_forSAHM_Raster
using code field as value, cell assign = max

In [132]:
print(code)
# Convert these points to a raster
template = r"D:\1_AMLT\1_SDM\SAHM\CovariatesFinal\bio_1_temp.tif"
pointras = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\All_" + code + "_forSAHM_Raster"

with arcpy.EnvManager(snapRaster=template, cellSize=template):
    arcpy.conversion.PointToRaster(allpoints, code, pointras, "MAXIMUM","NONE", template, "BUILD")

print('Raster created')

ARCT
Raster created


#### Raster to point (centroids) All_code_forSAHM_FromRaster
and number of remaining points

In [133]:
print(code)
# convert raster to points (centroids)
finalpoints = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\All_" + code + "_forSAHM_FromRaster"

arcpy.conversion.RasterToPoint(pointras, finalpoints)
print('Points from raster created')

# print number of records/points
result = arcpy.GetCount_management(finalpoints)
count = int(result.getOutput(0))
print('Total features in final point layer = ', count)

ARCT
Points from raster created
Total features in final point layer =  1447


#### Add fields for lat as y, long as x in WGS84, responseBinary with 1s, and species code column with 1s as well, export as csv
All_code_forSAHM_FromRaster.csv
Save the csv in this folder: D:\1_AMLT\1_SDM\SAHM\SpeciesLocations

In [2]:
finalpoints = r"D:\1_AMLT\1_SDM\SDM_Pro\SDM.gdb\All_Plants_forSAHM_FromRaster2"
code = 'PLAN'
print(code)
sppLocation = r"D:\1_AMLT\1_SDM\SAHM\SpeciesLocations"
csvfile = "All_" + code + "_forSAHM_FromRaster.csv"

# add field named as species code
arcpy.AddField_management(finalpoints, code, "LONG")
# set to one
arcpy.management.CalculateField(finalpoints, code, 1)

# add field named responseBinary
arcpy.AddField_management(finalpoints, 'resposeBinary', "LONG")
# set to one
arcpy.management.CalculateField(finalpoints, 'resposeBinary', 1)

# add fields x and y
arcpy.AddField_management(finalpoints, 'x', "DOUBLE")
arcpy.AddField_management(finalpoints, 'y', "DOUBLE")
# calculate lat and long coordinate values in WGS84
arcpy.management.CalculateGeometryAttributes(finalpoints, "x POINT_X;y POINT_Y", '', '', 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "DD")

print('Fields added')

# deleting two unnecessary fields
arcpy.DeleteField_management(finalpoints, ["grid_code", "pointid", "bio_1_temp_1"])

arcpy.TableToTable_conversion(finalpoints, sppLocation, csvfile)

print('Exported to csv')

PLAN
Fields added
Exported to csv
